In [27]:
# The code was removed by Watson Studio for sharing.

# Predicting English Premier League Matches

By: Yi Xuan Sim

## Model Deployment

We will now create a fixture table of all possible matches, predict the outcomes using a neural network, and save it as a csv file for deployment.

In [1]:
# The code was removed by Watson Studio for sharing.

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA
0,E0,10/08/2018,Man United,Leicester,2,1,H,1,0,H,...,1.79,17,-0.75,1.75,1.70,2.29,2.21,1.55,4.07,7.69
1,E0,11/08/2018,Bournemouth,Cardiff,2,0,H,1,0,H,...,1.83,20,-0.75,2.20,2.13,1.80,1.75,1.88,3.61,4.70
2,E0,11/08/2018,Fulham,Crystal Palace,0,2,A,0,1,A,...,1.87,22,-0.25,2.18,2.11,1.81,1.77,2.62,3.38,2.90
3,E0,11/08/2018,Huddersfield,Chelsea,0,3,A,0,2,A,...,1.84,23,1.00,1.84,1.80,2.13,2.06,7.24,3.95,1.58
4,E0,11/08/2018,Newcastle,Tottenham,1,2,A,1,2,A,...,1.81,20,0.25,2.20,2.12,1.80,1.76,4.74,3.53,1.89


In [3]:
import numpy as np

#Creating a list of teamnames
homegroup = data_222.groupby('HomeTeam')
teamnames = homegroup.HomeTeam.all().values #copy the home team name 
teamnames = list(sorted(set(teamnames)))
full_list = np.repeat(teamnames,19)

new_list=[]
#Make new list for away fixtures
for i in range(20):
    new_list.append(teamnames)
flat_list = [item for sublist in new_list for item in sublist]

pop_count = 0

#we remove elements
for i in range(20):
    flat_list.pop(pop_count)
    pop_count = pop_count + 20


In [4]:
data_333 = pd.DataFrame()
data_333['HomeTeam'] = full_list
data_333['AwayTeam'] = flat_list


In [5]:
data_333.head()

,HomeTeam,AwayTeam
0,Arsenal,Bournemouth
1,Arsenal,Brighton
2,Arsenal,Burnley
3,Arsenal,Cardiff
4,Arsenal,Chelsea


data_333 is the fixture table containing all possible matches in the 2018-2019 season. We will now append the calculated team scores, pass the features into a neural network, and obtain the output. 

In [8]:

body = client_0550608025c545658c8a0d3996cfa335.get_object(Bucket='courseracapstone-donotdelete-pr-otnumxi5bu7tbr',Key='team_scores2.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

team_scores = pd.read_csv(body)
team_scores.head()



,teamnames,n_homegoals,n_homeconcede,n_awaygoals,n_awayconcede,n_hst,n_hsto,n_ast,n_asto,n_homematch,n_awaymatch,home_attack_score,away_attack_score,home_defense_score,away_defense_score,home_attack_efficiency,away_attack_efficiency,home_inv_defense_efficiency,away_inv_defense_efficiency
0,Arsenal,566.0,208.0,420.0,333.0,2090.0,994.0,1585.0,1289.0,258,257,2.117749,2.010511,-1.481912,-1.247002,0.548548,0.874321,-0.696817,-0.004143
1,Aston Villa,256.0,251.0,239.0,338.0,1126.0,1207.0,1006.0,1472.0,209,209,-0.367044,0.453356,-0.143126,-0.189184,-0.241390,0.318137,-0.722440,-0.633718
2,Birmingham,87.0,69.0,63.0,137.0,425.0,448.0,372.0,646.0,76,76,-0.572572,-0.544953,-1.137026,0.420843,-0.653038,-1.066144,-1.783932,-1.018328
3,Blackburn,192.0,153.0,131.0,239.0,871.0,756.0,690.0,995.0,133,133,0.193886,-0.049866,-0.314668,0.402289,-0.367118,-0.650164,-0.832113,-0.401763
4,Bolton,174.0,173.0,129.0,238.0,964.0,898.0,648.0,1034.0,133,133,-0.153189,-0.097585,0.195321,0.377551,-1.093047,-0.463096,-1.023619,-0.621563


In [10]:
#Transforming into data with suitable input features
# The attacking and defending scores for each team that was calculated previously is applied to the dataframe. 
home_attack_score_analysis = []  
away_attack_score_analysis =[]
home_defense_score_analysis = []
away_defense_score_analysis = []
home_attack_efficiency_analysis = []  
away_attack_efficiency_analysis =[]
home_inv_defense_efficiency_analysis = []
away_inv_defense_efficiency_analysis = []

#this is to apply the attacking and defensive score of each team from Table_1 into each match in Table_2.
for index,row in data_333.iterrows():
    home_attack_score_analysis.append(team_scores[team_scores['teamnames'] == row['HomeTeam']]['home_attack_score'].values[0])
    away_attack_score_analysis.append(team_scores[team_scores['teamnames'] == row['AwayTeam']]['away_attack_score'].values[0])
    home_defense_score_analysis.append(team_scores[team_scores['teamnames'] == row['HomeTeam']]['home_defense_score'].values[0])
    away_defense_score_analysis.append(team_scores[team_scores['teamnames'] == row['AwayTeam']]['away_defense_score'].values[0])
    home_attack_efficiency_analysis.append(team_scores[team_scores['teamnames'] == row['HomeTeam']]['home_attack_efficiency'].values[0])
    away_attack_efficiency_analysis.append(team_scores[team_scores['teamnames'] == row['AwayTeam']]['away_attack_efficiency'].values[0])
    home_inv_defense_efficiency_analysis.append(team_scores[team_scores['teamnames'] == row['HomeTeam']]['home_inv_defense_efficiency'].values[0])
    away_inv_defense_efficiency_analysis.append(team_scores[team_scores['teamnames'] == row['AwayTeam']]['away_inv_defense_efficiency'].values[0])
    
#creating new columns for the scores.
data_333['home_attack_score_analysis'] = home_attack_score_analysis
data_333['away_attack_score_analysis'] = away_attack_score_analysis
data_333['home_defense_score_analysis'] = home_defense_score_analysis
data_333['away_defense_score_analysis'] = away_defense_score_analysis
data_333['home_attack_efficiency_analysis'] = home_attack_efficiency_analysis
data_333['away_attack_efficiency_analysis'] = away_attack_efficiency_analysis
data_333['home_inv_defense_efficiency_analysis'] = home_inv_defense_efficiency_analysis
data_333['away_inv_defense_efficiency_analysis'] = away_inv_defense_efficiency_analysis

data_333.head()

,HomeTeam,AwayTeam,home_attack_score_analysis,away_attack_score_analysis,home_defense_score_analysis,away_defense_score_analysis,home_attack_efficiency_analysis,away_attack_efficiency_analysis,home_inv_defense_efficiency_analysis,away_inv_defense_efficiency_analysis
0,Arsenal,Bournemouth,2.117749,0.282032,-1.481912,0.628258,0.548548,1.593195,-0.696817,2.561392
1,Arsenal,Brighton,2.117749,-1.165701,-1.481912,-0.355531,0.548548,0.316625,-0.696817,1.201370
2,Arsenal,Burnley,2.117749,-0.481140,-1.481912,1.163346,0.548548,-0.154429,-0.696817,0.625200
3,Arsenal,Cardiff,2.117749,-1.205824,-1.481912,0.956771,0.548548,-0.048375,-0.696817,1.274440
4,Arsenal,Chelsea,2.117749,1.973468,-1.481912,-2.437623,0.548548,0.707711,-0.696817,-1.333567


In [11]:
#Difference in attacking score
x = data_333['home_attack_score_analysis']
y = data_333['away_attack_score_analysis'] 
data_333["diff_attack"] = x-y 

#Difference in defending score
x = data_333['away_defense_score_analysis']
y = data_333['home_defense_score_analysis'] 
data_333["diff_defense"] = x-y

#Overall Score
data_333["overall_score"] = data_333['diff_attack'] + data_333['diff_defense']
data_333.head()

,HomeTeam,AwayTeam,home_attack_score_analysis,away_attack_score_analysis,home_defense_score_analysis,away_defense_score_analysis,home_attack_efficiency_analysis,away_attack_efficiency_analysis,home_inv_defense_efficiency_analysis,away_inv_defense_efficiency_analysis,diff_attack,diff_defense,overall_score
0,Arsenal,Bournemouth,2.117749,0.282032,-1.481912,0.628258,0.548548,1.593195,-0.696817,2.561392,1.835717,2.110170,3.945887
1,Arsenal,Brighton,2.117749,-1.165701,-1.481912,-0.355531,0.548548,0.316625,-0.696817,1.201370,3.283451,1.126381,4.409832
2,Arsenal,Burnley,2.117749,-0.481140,-1.481912,1.163346,0.548548,-0.154429,-0.696817,0.625200,2.598889,2.645258,5.244147
3,Arsenal,Cardiff,2.117749,-1.205824,-1.481912,0.956771,0.548548,-0.048375,-0.696817,1.274440,3.323573,2.438683,5.762256
4,Arsenal,Chelsea,2.117749,1.973468,-1.481912,-2.437623,0.548548,0.707711,-0.696817,-1.333567,0.144281,-0.955711,-0.811430


### Training Neural Network
Firstly, we must train the neural network using previous data. 

In [14]:

body = client_0550608025c545658c8a0d3996cfa335.get_object(Bucket='courseracapstone-donotdelete-pr-otnumxi5bu7tbr',Key='training_scores2.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()



,HomeTeam,AwayTeam,FTR,home_attack_score_analysis,away_attack_score_analysis,home_defense_score_analysis,away_defense_score_analysis,home_attack_efficiency_analysis,away_attack_efficiency_analysis,home_inv_defense_efficiency_analysis,away_inv_defense_efficiency_analysis,diff_attack,diff_defense,overall_score
0,Aston Villa,Bolton,D,-0.367044,-0.097585,-0.143126,0.377551,-0.241390,-0.463096,-0.722440,-0.621563,-0.269459,0.520677,0.251218
1,Everton,Man United,A,0.855378,2.162607,-0.614336,-2.168971,0.418593,0.998026,-0.438074,-1.517423,-1.307229,-1.554635,-2.861864
2,Fulham,Birmingham,D,0.076383,-0.544953,0.074727,0.420843,-0.483306,-1.066144,-0.439501,-1.018328,0.621337,0.346116,0.967453
3,Man City,West Brom,D,2.167449,-0.614544,-1.192720,-0.411329,1.120426,-0.396703,-0.398956,-0.544577,2.781993,0.781391,3.563383
4,Middlesbrough,Liverpool,D,-0.268881,1.670617,0.067824,-1.633356,-0.334656,0.369332,-0.166129,-0.043594,-1.939499,-1.701180,-3.640678


Neural networks only accept numerical inputs and outputs, thus we use one hot encoding to transform the labels

We first tranform the labels into integers. Later we will use keras to_categorical to transform into binary values.

The encoding is as follows: <br/>
Win by away team = 0 <br/>
Draw = 1 <br/>
Win by home team = 2 <br/> 

In [15]:
full_time = df_data_1['FTR']
analysis_NN = df_data_1.copy()

from sklearn.preprocessing import LabelEncoder
# integer encode
label_encoder = LabelEncoder()
FT_encoded = label_encoder.fit_transform(full_time)
analysis_NN['FT_encoded'] = FT_encoded

#drop all non numerical values
analysis_NN = analysis_NN.drop(['HomeTeam','AwayTeam','FTR'], axis=1)
analysis_NN.head()

,home_attack_score_analysis,away_attack_score_analysis,home_defense_score_analysis,away_defense_score_analysis,home_attack_efficiency_analysis,away_attack_efficiency_analysis,home_inv_defense_efficiency_analysis,away_inv_defense_efficiency_analysis,diff_attack,diff_defense,overall_score,FT_encoded
0,-0.367044,-0.097585,-0.143126,0.377551,-0.241390,-0.463096,-0.722440,-0.621563,-0.269459,0.520677,0.251218,1
1,0.855378,2.162607,-0.614336,-2.168971,0.418593,0.998026,-0.438074,-1.517423,-1.307229,-1.554635,-2.861864,0
2,0.076383,-0.544953,0.074727,0.420843,-0.483306,-1.066144,-0.439501,-1.018328,0.621337,0.346116,0.967453,1
3,2.167449,-0.614544,-1.192720,-0.411329,1.120426,-0.396703,-0.398956,-0.544577,2.781993,0.781391,3.563383,1
4,-0.268881,1.670617,0.067824,-1.633356,-0.334656,0.369332,-0.166129,-0.043594,-1.939499,-1.701180,-3.640678,1


In [19]:
from keras.utils import to_categorical
x_all = analysis_NN.drop(['FT_encoded'], axis=1)
#to_categorical converts integers into binary values
y_all = to_categorical(analysis_NN['FT_encoded'])
print(analysis_NN.shape[0])

5150


The Neural Network that we are using has been defined in the previous step (Model definition)

In [21]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from keras import metrics

model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
model.add(Dense(64, input_dim=11))
model.add(Activation('sigmoid'))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(3, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(np.array(x_all), np.array(y_all),epochs=20, batch_size=128, validation_split=0.3)

Train on 3604 samples, validate on 1546 samples
Epoch 1/20
3604/3604 [==============================] - 0s - loss: 1.1099 - acc: 0.3976 - val_loss: 1.0780 - val_acc: 0.4541
Epoch 2/20
3604/3604 [==============================] - 0s - loss: 1.0555 - acc: 0.4731 - val_loss: 1.0598 - val_acc: 0.4541
Epoch 3/20
3604/3604 [==============================] - 0s - loss: 1.0505 - acc: 0.4731 - val_loss: 1.0566 - val_acc: 0.4541
Epoch 4/20
3604/3604 [==============================] - 0s - loss: 1.0470 - acc: 0.4731 - val_loss: 1.0547 - val_acc: 0.4541
Epoch 5/20
3604/3604 [==============================] - 0s - loss: 1.0424 - acc: 0.4731 - val_loss: 1.0493 - val_acc: 0.4541
Epoch 6/20
3604/3604 [==============================] - 0s - loss: 1.0383 - acc: 0.4731 - val_loss: 1.0448 - val_acc: 0.4541
Epoch 7/20
3604/3604 [==============================] - 0s - loss: 1.0331 - acc: 0.4731 - val_loss: 1.0398 - val_acc: 0.4541
Epoch 8/20
3604/3604 [==============================] - 0s - loss: 1.0267 - a

### Predicting the outcomes

In [22]:
predict_NN =data_333.copy()
x_predict = predict_NN.drop(['HomeTeam','AwayTeam'], axis=1)
y_predict = model.predict(np.array(x_predict))

In [23]:
print(y_predict)

[[ 0.1035757   0.17404416  0.72238016]
 [ 0.09512789  0.16552511  0.73934704]
 [ 0.08968587  0.16047519  0.74983889]
 ..., 
 [ 0.52316743  0.28831372  0.18851881]
 [ 0.39110735  0.29489869  0.31399393]
 [ 0.44946089  0.29462588  0.25591317]]


In [24]:
y_pred_int = np.argmax(y_predict, axis=1)

#Encode back into ADH form
y_pred_encode = pd.cut(y_pred_int,3, labels=["A", "D", "H"])


In [25]:
threshold = 0.05
count_1 = 0
for row in y_predict:
    diff = abs(row[2] - row[0])
    if (diff<threshold):
        y_pred_encode[count_1] = 'D'
    count_1 += 1

In [26]:
final_predict_NN = data_333[['HomeTeam','AwayTeam']]
final_predict_NN['y_pred'] = y_pred_encode
final_predict_NN.head()

/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,HomeTeam,AwayTeam,y_pred
0,Arsenal,Bournemouth,H
1,Arsenal,Brighton,H
2,Arsenal,Burnley,H
3,Arsenal,Cardiff,H
4,Arsenal,Chelsea,H


### Saving Results as CSV file for deployment
We now have a dataframe containing all possible matches in this seasons as well as their predicted outcomes. We will now save it into a csv file. 

In [29]:
#Saving as csv
project.save_data(data=final_predict_NN.to_csv(index=False),file_name='fixtures.csv',overwrite=True)

{'asset_id': '05307a2d-c1c7-4767-8e56-c2a916b9020b',
 'bucket_name': 'courseracapstone-donotdelete-pr-otnumxi5bu7tbr',
 'file_name': 'fixtures.csv',
 'message': 'File fixtures.csv has been written successfully to the associated OS'}